In [17]:
import shap
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl

In [64]:
#import necessary libraries
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from pycox.models import CoxPH, DeepHit

from pycox.evaluation import EvalSurv
from pycox.simulations import SimStudyLinearPH

import torch
import torchtuples as tt

In [71]:
url = 'https://raw.githubusercontent.com/camicallierotti/imperial-summer-project/main/pbc.csv'
df = pd.read_csv(url, sep=";", encoding='latin1',engine='python', header=0, decimal=',')
df = pd.DataFrame(df)

In [72]:
df.head()

,id,time,status,trt,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk.phos,ast,trig,platelet,protime,stage
0,1,400,2,D-penicillmain,58.765229,female,yes,yes,yes,edema,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,IV
1,2,4500,0,D-penicillmain,56.446270,female,no,yes,yes,no,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,III
2,3,1012,2,D-penicillmain,70.072553,male,no,no,no,untreated,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,IV
3,4,1925,2,D-penicillmain,54.740589,female,no,yes,yes,untreated,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,IV
4,5,1504,1,Placebo,38.105407,female,no,yes,yes,no,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,III


In [75]:
# convert to single outcome

df = df.applymap(str) 
df["status"].replace({"1": "0", "2": "1"}, inplace=True)

In [76]:
# drop missing values from the dataframe
df_new = df
df_new.dropna(how='any', inplace= True)

## create X

In [81]:
X = df_new.drop(['id', 'time', 'status'], axis=1)

In [82]:
X.dtypes

trt         object
age         object
sex         object
ascites     object
hepato      object
spiders     object
edema       object
bili        object
chol        object
albumin     object
copper      object
alk.phos    object
ast         object
trig        object
platelet    object
protime     object
stage       object
dtype: object

In [83]:
list(X)

['trt',
 'age',
 'sex',
 'ascites',
 'hepato',
 'spiders',
 'edema',
 'bili',
 'chol',
 'albumin',
 'copper',
 'alk.phos',
 'ast',
 'trig',
 'platelet',
 'protime',
 'stage']

In [84]:
# fix object data types
obj_cols = ['trt',
 'age',
 'sex',
 'ascites',
 'hepato',
 'spiders',
 'edema',
 'bili',
 'chol',
 'albumin',
 'copper',
 'alk.phos',
 'ast',
 'trig',
 'platelet',
 'protime',
 'stage']
LE = LabelEncoder()
for col in obj_cols:
    X[col] = lbl.fit_transform(X[col].astype(str))


## create y (structured array)

In [85]:
y = df_new[['status', 'time']]

In [86]:
y_str = y.to_records(index=False)
print(y_str)

[('1', '400') ('0', '4500') ('1', '1012') ('1', '1925') ('0', '1504')
 ('1', '2503') ('0', '1832') ('1', '2466') ('1', '2400') ('1', '51')
 ('1', '3762') ('1', '304') ('0', '3577') ('1', '1217') ('1', '3584')
 ('0', '3672') ('1', '769') ('1', '131') ('0', '4232') ('1', '1356')
 ('0', '3445') ('1', '673') ('1', '264') ('1', '4079') ('0', '4127')
 ('1', '1444') ('1', '77') ('1', '549') ('0', '4509') ('1', '321')
 ('1', '3839') ('0', '4523') ('1', '3170') ('0', '3933') ('1', '2847')
 ('0', '3611') ('1', '223') ('1', '3244') ('1', '2297') ('0', '4467')
 ('1', '1350') ('0', '4453') ('0', '4556') ('1', '3428') ('0', '4025')
 ('1', '2256') ('0', '2576') ('0', '4427') ('1', '708') ('1', '2598')
 ('1', '3853') ('1', '2386') ('1', '1000') ('1', '1434') ('1', '1360')
 ('1', '1847') ('1', '3282') ('0', '4459') ('1', '2224') ('0', '4365')
 ('0', '4256') ('1', '3090') ('1', '859') ('1', '1487') ('0', '3992')
 ('1', '4191') ('1', '2769') ('0', '4039') ('1', '1170') ('0', '3458')
 ('0', '4196') ('0', 

In [87]:
xgb_full = xgboost.DMatrix(X, label=y)

AssertionError: 